# Settings

In [11]:
import os
import sys

import pandas as pd
import numpy as np

import lxml

np.random.seed(42)
pd.set_option('display.max_columns', 500)

In [4]:
os.chdir('..')

In [17]:
DATA_DIR = './data/'
MATCH_DIR = os.path.join(DATA_DIR, 'French Ligue One 20162017 season - Match Day 1- 19/')
ID_PATH = os.path.join(DATA_DIR, 'Noms des joueurs et IDs - F40 - L1 20162017.xml')

# Definition

In [319]:
class League:
    
    def __init__(self, file):
        root = lxml.etree.parse(file).getroot()
        self._doc = root.getchildren()[0] # only one element in the list
        self._params = {}
        for k, v in self._doc.items():
            self._params.update({k: v})
        self.name=self._params['competition_name']
        self.season=self._params['season_id']
        self.season_name=self._params['season_name']
        
        self._club = {}
        self._club_id = {}
        for c in self._doc.findall('Team'):
            club = Club(c)
            if not club.name is None:
                self._club[club.name] = club
                #self._club_id[club.uID] = club.name

            
    def get_club(self, clubname):
        return self._club.get(clubname)
    
    def get_info(self):
        params_dict = []
        for _, club in self._club.items():
            params_dict.append(club._params)
        return pd.DataFrame(params_dict).set_index('uID')
    
    def display(self):
        for k, v in self._params.items():
            print(k, ':', v)
            

class Club:
    
    def __init__(self, etree):
        self._etree=etree
        self._params = {}
        self._params['founded'] = self._etree.findall('Founded')[0].text

        for k, v in etree.items():
            self._params.update({k: v})
        if len(self._params.keys())>0:
            self.name=self._params['short_club_name']
            self.uID=self._params['uID']
        else:
            self.name=None
        
        self._params['symid'] = self._etree.findall('SYMID')[0].text
        
        self._player={}
        for p in self._etree.findall('Player'):
            player = Player(p)
            self._player[player.name] = player
            
    def get_player(self, playername):
        return self._player.get(playername)
    
    def get_info(self):
        params_dict = []
        for _, player in self._player.items():
            params_dict.append(player._params)
        df=pd.DataFrame(params_dict).set_index('uID')
        df['tID']=self.uID
        return df
    
    def display(self):
        for k, v in self._params.items():
            print(k, ':', v)
    
class Player:
    
    def __init__(self, etree):
        self._etree=etree
        self._params={}
        self._params.update(dict(self._etree.items()))
        self._params['name'] = self._etree.findall('Name')[0].text
        self.name = self._params['name']
        self._params['position'] = self._etree.findall('Position')[0].text
        for c in self._etree.findall('Stat'):
            _, stat = c.items()[0]
            self._params[stat] = c.text

            
    def display(self):
        for k, v in self._params.items():
            print(k, ':', v)
        
        
class TeamOfficial:
    # President, Manager
    
    def __init__(self, etree):
        pass
    
class TeamKits:
    
    def __init__(self, etree):
        pass
       


# Loader

In [320]:
league = League(ID_PATH)

In [321]:
league.get_info()

,city,country,country_id,country_iso,founded,postal_code,region_id,region_name,short_club_name,street,symid,web_address
uID,,,,,,,,,,,,
t2128,NaN,France,8,FR,1919,NaN,17,Europe,Angers,NaN,ANR,www.angers-sco.fr
t139,NaN,France,8,FR,1905,NaN,17,Europe,Bastia,NaN,BAS,www.sc-bastia.net
t140,Bordeaux,France,8,FR,1881,33 187,17,Europe,Bordeaux,Rue Joliot Curie - Le Haillan,BOR,www.girondins.com
t1028,NaN,France,8,FR,1913,NaN,17,Europe,Caen,NaN,CAE,www.smcaen.fr
t2130,NaN,France,8,FR,1998,NaN,17,Europe,Dijon,NaN,DIJ,www.dfco.fr
t428,NaN,France,8,FR,1912,NaN,17,Europe,Guingamp,NaN,GUI,www.eaguingamp.com/
t429,NaN,France,8,FR,1944,NaN,17,Europe,Lille,NaN,LIL,www.losc.fr
t694,NaN,France,8,FR,1926,NaN,17,Europe,Lorient,NaN,LOR,www.fclweb.fr
t143,Lyon,France,8,FR,1950,69007,17,Europe,Lyon,350 Avenue Jean-Jaurès,LYN,www.olweb.fr


In [322]:
nancy = league.get_club('Nancy')

In [323]:
nancy.display()

founded : 1967
country : France
country_id : 8
country_iso : FR
region_id : 17
region_name : Europe
short_club_name : Nancy
uID : t148
web_address : www.asnl.net
symid : NAC


In [324]:
nancy.get_info()

,birth_date,birth_place,country,first_name,first_nationality,height,jersey_num,join_date,known_name,last_name,loan,middle_name,name,position,preferred_foot,real_position,real_position_side,weight,tID
uID,,,,,,,,,,,,,,,,,,,
p63600,1988-07-20,Grodno,Belarus,Sergey,Belarus,187,1,2016-07-01,NaN,Chernik,NaN,NaN,Sergey Chernik,Goalkeeper,NaN,Goalkeeper,Unknown,84,t148
p38816,1986-02-28,Yaoundé,Cameroon,Guy,Cameroon,185,16,2011-08-02,NaN,NDy Assembe,NaN,Roland,Guy NDy Assembe,Goalkeeper,NaN,Goalkeeper,Unknown,88,t148
p174598,1996-07-05,NaN,France,Alexandre,NaN,186,30,2013-08-31,NaN,Menay,NaN,NaN,Alexandre Menay,Goalkeeper,NaN,Goalkeeper,Unknown,76,t148
p152337,1993-05-12,Dijon,Congo,Tobias,France,181,3,2013-01-22,NaN,Badila,NaN,NaN,Tobias Badila,Defender,NaN,Full Back,Left,80,t148
p170034,1994-01-03,NaN,Senegal,Modou,France,188,4,2013-08-30,NaN,Diagne,NaN,NaN,Modou Diagne,Defender,NaN,Central Defender,Centre,74,t148
p247120,1997-04-29,Rambouillet,France,Ryan,France,185,5,2016-07-01,NaN,Bidounga,NaN,NaN,Ryan Bidounga,Defender,NaN,Central Defender,Centre,83,t148
p80799,1988-03-15,Méru,France,Joffrey,France,180,14,2013-07-01,NaN,Cuffaut,NaN,NaN,Joffrey Cuffaut,Defender,NaN,Full Back,Right,74,t148
p49169,1984-07-10,Nancy,Morocco,Michael Basser,France,179,20,2015-07-24,NaN,Chretien,NaN,NaN,Michael Basser Chretien,Defender,NaN,Full Back,Left/Right,70,t148
p169586,1995-04-19,Montevideo,None,Erick,Uruguay,185,24,2016-08-27,NaN,Cabaco,1,NaN,Erick Cabaco,Defender,Right,Central Defender,Centre,84,t148


In [317]:
pedretti = nancy.get_player('Benoit Pedretti')

In [318]:
pedretti.display()

uID : p9980
name : Benoit Pedretti
position : Midfielder
first_name : Benoit
last_name : Pedretti
birth_date : 1980-11-12
birth_place : Audincourt
first_nationality : France
weight : 71
height : 178
jersey_num : 25
real_position : Central Midfielder
real_position_side : Centre
join_date : 2015-08-01
country : France
